In [134]:
#Import necessary library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [135]:
from numpy.lib.shape_base import vsplit
#Export data

filename = 'psi-outd1.dat'
data = np.loadtxt(filename)

#Make list to store information from data
x = []        
psi = []
psi2 = []
xpsi2 = []
x2psi2 = []
potv = []

for i in range(0, len(data)):
  x.append(data[i][0])
  psi.append(data[i][1])
  psi2.append(data[i][1]*data[i][1])
  xpsi2.append(data[i][0]*psi2[i])
  x2psi2.append(data[i][0]*data[i][0]*psi2[i])

#Set the potential (based on potential used to calculate data)
for i in range(0, len(x)):
  #if statement
  v = 3.5
  potv.append(v)

In [136]:
def integrate(xpos, to_integrate):
  '''
  A function to integrate a list using Simpson 1/3 integration method
  Input   : a list of x data, a list of quantity to integrate
  Return  : intergartion result
  '''

  xmin = min(xpos)
  h = abs(x[2]-x[1])/2
  dx = abs(x[2]-x[1])
  result = 0

  for i in range(len(xpos)-2):
    result = result + (to_integrate[i] + 4*to_integrate[i+1] + to_integrate[i+2])
    xmin = xmin + dx 

  result = result * h/3

  return result

In [137]:
def diff(to_derive):
  '''
  Calculates derivation of points in data using forward finite
  Input   : a list of data to derive
  Return  : a list of derivation of each points in data
  '''

  result = []
  result.append(0) #Initialize first element
  dx = 0.01
  for i in range(0, len(to_derive)-1):
    df = (to_derive[i+1] - to_derive[i-1])/2*dx
    result.append(df)

  return result

In [138]:
def diff2(to_derive):
  '''
  Calculates second derivation of points in data using second order forward finite
  Input   : a list of data to derive
  Return  : a list of derivation of each points in data
  '''

  result = []
  result.append(0) #Initialize first element
  dx = 0.01
  for i in range(0, len(to_derive)-1):
    df2 = (to_derive[i+1]-2*to_derive[i]+to_derive[i-1])/(dx**2)
    result.append(df2)

  return result

In [139]:
def probability(xpos, psi2):
  '''
  Calculates the probability
  Input   : a list of x data, a list of psi*psi data
  Return  : probability (must equal to 1)
  '''

  prob = integrate(xpos, psi2)
  return prob

def position_xv(xpos, xpsi2):
  '''
  Calculates position expectation value
  Input   : a list of x data, a list of  x*psi*psi data
  Return  : position expectation value
  '''
  pos_xv = integrate(xpos, xpsi2)
  return pos_xv

def squarepos_xv(xpos, x2psi2):
  '''
  Calculates squared position expectation value
  Input   : a list of x data, a list of  x*x*psi*psi data
  Return  : squared position expectation value
  '''
  spos_xv = integrate(xpos, x2psi2)
  return spos_xv

def momentum_xv(xpos, psi):
  '''
  Calculates momentum expectation value
  Input   : a list of x data, a list of  psi data
  Return  : momentum expectation value
  '''

  df = diff(psi)
  dpsi = [psi[i] * df[i] for i in range(len(psi))]
  mom_xv = integrate(xpos, dpsi)
  return mom_xv

def squaremom_xv(xpos, psi):
  '''
  Calculates squared momentum expectation value
  Input   : a list of x data, a list of  psi data
  Return  : squared momentum expectation value
  '''

  df2 = diff2(psi)
  dpsi2 = [psi[i] * df2[i] for i in range(len(psi))]
  smom_xv = -1/4 * integrate(xpos, dpsi2)
  return smom_xv

def potential(xpos, psi2, pot):
  '''
  Calculates potential
  Input   : a list of x data, a list of  psi*psi data, a list of potential data
  Return  : potential value
  '''
  
  vpsi2 = [pot[i] * psi2[i] for i in range(len(psi2))]
  potv = integrate(xpos, vpsi2)
  return potv

def kinetic(xpos, psi):
  '''
  Calculates kinetic
  Input   : a list of x data, a list of  psi data
  Return  : kinetic value
  '''

  k = 0.5 * squaremom_xv(xpos, psi)
  return k


In [140]:
def analytics(L, n):
  '''
  Calculate analytical expectatation value for symmetric well
  Input   : well width, energy level
  Return  : analytical value
  '''
  
  # hbar = 1.04e-34
  hbar = 1
  prob = 1
  pos_xv = 0
  spos_xv = (L/2)**2 * (1/3 - 2/((n*np.pi)**2))
  mom_xv = 0
  smom_xv = ((n**2 * np.pi**2 * hbar**2)/(L**2)) #* 4
  energy = ((n**2 * np.pi**2 * hbar**2) / (2*L)) + v0
  tp_pos = (L/2) * (np.sqrt((1/3) - 2/(n*np.pi)**2))
  tp_mom = (n*np.pi*hbar)/L
  # tp_pos = np.sqrt(spos_xv - (pos_xv**2))
  # tp_mom = np.sqrt(smom_xv - (mom_xv**2))
  # tp_heis = (hbar/2) * (np.sqrt((n**2 * np.pi**2)/3)-2)
  tp_heis = tp_pos * tp_mom
  
  return prob, pos_xv, spos_xv, mom_xv, smom_xv, energy, tp_pos, tp_mom, tp_heis

In [141]:
L = abs(min(x)*2)
# print(L)
n = 1
v0 = 3.5

prob_a, pos_xv_a, spos_xv_a, mom_xv_a, smom_xv_a, energy_a, tp_pos_a, tp_mom_a, tp_heis_a = analytics(L, n)
prob = probability(x, psi2)
pos_xv = position_xv(x, xpsi2)
spos_xv = squarepos_xv(x, x2psi2)
mom_xv = momentum_xv(x, psi)
smom_xv = squaremom_xv(x, psi)
energy = kinetic(x, psi) + potential(x, psi2, potv)
tp_pos = np.sqrt(spos_xv - (pos_xv**2))
tp_mom = np.sqrt(smom_xv - (mom_xv**2))
tp_heis = tp_pos * tp_mom

#Make dataframe to store values
expectation_value = ['Probability', 'Position', 'Square position', 'Momentum', 'Squared Momentum', 
                     'Energy', 'Position Uncertainty', 'Momentum Uncertainty', 'Heisenberg Uncertainty']
output = pd.DataFrame(columns=['Expectation Value', 'Numerical', 'Analytical'])
output.loc[0] = [expectation_value[0], abs(prob), prob_a]
output.loc[1] = [expectation_value[1], pos_xv, pos_xv_a]
output.loc[2] = [expectation_value[2], spos_xv, spos_xv_a]
output.loc[3] = [expectation_value[3], mom_xv, mom_xv_a]
output.loc[4] = [expectation_value[4], smom_xv, smom_xv_a]
output.loc[5] = [expectation_value[5], energy, energy_a]
output.loc[6] = [expectation_value[6], tp_pos, tp_pos_a]
output.loc[7] = [expectation_value[7], tp_mom, tp_mom_a]
output.loc[8] = [expectation_value[8], tp_heis, tp_heis_a]
output

,Expectation Value,Numerical,Analytical
0,Probability,1.000035,1
1,Position,-0.000015,0
2,Square position,0.522772,0.522764
3,Momentum,0.000002,0
4,Squared Momentum,0.611660,0.61685
5,Energy,3.805953,4.733701
6,Position Uncertainty,0.723030,0.723024
7,Momentum Uncertainty,0.782087,0.785398
8,Heisenberg Uncertainty,0.565472,0.567862
